In [1]:
from _plotly_future_ import v4_subplots

import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
from ipywidgets import interactive, HBox, VBox

from plotly.offline import *

In [2]:
from plotly.subplots import make_subplots
from plotly import tools

clasificar_j = pd.read_csv('ENORTE.csv', delimiter = ',') ## Import the dataset desde sklearn.datasets
print(clasificar_j.columns)
clasificar_j.dtypes
clasificar_j.head()

Index(['s1', 's3', 'x', 'y', 'z', 'MINA', 'LIT', 'ALT', 'MIN', 'UNIDAD',
       'TODOLIX', 'PRIMSEC', 'SEC', 'PRIM'],
      dtype='object')


,s1,s3,x,y,z,MINA,LIT,ALT,MIN,UNIDAD,TODOLIX,PRIMSEC,SEC,PRIM
0,0.0,-3.934700,17798.655,114150.946,3138.480,ESCN,AND,QSA,LIX,TODOLIX,0.0,NaN,NaN,NaN
1,0.0,-3.919300,19213.348,113566.638,3060.069,ESCN,AND,QSA,LIX,TODOLIX,0.0,NaN,NaN,NaN
2,0.0,-3.896200,18656.199,112906.778,3068.462,ESCN,AND,QSA,LIX,TODOLIX,0.0,NaN,NaN,NaN
3,0.0,-2.579500,18594.988,113894.293,3087.932,ESCN,AND,QSA,LIX,TODOLIX,0.0,NaN,NaN,NaN
4,0.0,-2.440086,17989.647,114682.192,3169.691,ESCN,AND,SCC,LIX,TODOLIX,0.0,NaN,NaN,NaN


In [3]:

init_notebook_mode(connected=True)

t = np.linspace(-1,1.2,2000)
x = clasificar_j['s3']
y = clasificar_j['s1']


trace1 = go.Scatter(
    x=x, y=y, mode='markers', name='Stress',
    marker=dict(color='#2C36CA', size=10, opacity=0.7, line=dict(width=1, color='DarkSlateGrey'))
)

trace2 = go.Histogram(
    y=y, name='UCS density', marker=dict(color='orange'), xaxis='x2',
    yaxis='y2', opacity=0.6)

#fig = make_subplots(rows=1, cols=2)

data = [trace1, trace2]

#fig.append_trace(trace1, 1, 1)
#fig.append_trace(trace2, 1, 2)

layout = go.Layout(
    showlegend=True,
    #autosize=True,
    xaxis=dict(
        title='Sigma 3 (Mpa)',
        titlefont=dict(
        family='Arial, sans-serif',
        size=18,
        color='lightgrey'),
        domain=[0.4, 1],
        showgrid=True,
        zeroline=True,
        gridcolor='#bdbdbd',
        gridwidth=0.7
    ),
    yaxis=dict(
        title='Sigma 1 (Mpa)',
        titlefont=dict(
        family='Arial, sans-serif',
        size=18,
        color='lightgrey'),
        domain=[0, 1],
        showgrid=True,
        zeroline=True,
        gridcolor='#bdbdbd',
        gridwidth=0.7,
        range=[0, 350]
    ),
    #margin=dict(
    #    t=50
    #),
    bargap=0.1,
    xaxis2=dict(
        domain=[0, 0.20],
        showgrid=True,
        zeroline=True,
        range=[30, 0]
    ),
    yaxis2=dict(
        domain=[0, 1],
        anchor='x2',
        range=[0, 350]
    )

)

fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig['layout'].update(height=600, width=800, title='Plot Awards',
                    annotations=[go.layout.Annotation(x=1.25, y=0.75, showarrow=False, text="Custom x-axis title",
                    bordercolor="#ff7f0e",
                    borderwidth=2,
                    xref="paper",
                    yref="paper")])
#fig.update_xaxis2(list(autorange = "reversed"))

py.iplot(fig, filename='multiple-subplots')


In [92]:
from ipywidgets import widgets
from IPython.display import clear_output
# Install a conda package in the current Jupyter kernel
#import sys
#!conda install --yes --prefix {sys.prefix} geopandas
import geopandas as gpd
import matplotlib.pyplot as plt



def SetColor(x):
    if(x == 0):
        return "blue"
    elif(x == 1):
        return "red"
    
def NameUG(x):
    if(x == 0):
        return "Used Data"
    elif(x == 1):
        return "Discarded"
    
def UCSUG(x):
    if(x == 0):
        return "Used UCS"
    elif(x == 1):
        return "Discarded UCS"


## definicion del grafico mediante plotly, la variable a cambiar corresponde al "hue"

def plotpro(jk):

    fig = make_subplots(rows=2, cols=3, specs=[[{"type": "scatter"},None, {"type": "bar"}],
                                               [{"type": "table"}, None, {"type": "scatter"}]], column_widths=[0.2, 0.1, 0.7], row_heights=[1, 1])


    for UG, stress in clasificar_j.groupby(jk):
        fig.add_trace(go.Scatter(x=stress.s3, y=stress.s1, name=NameUG(UG), mode='markers',
                                 marker=dict(size=10, opacity=0.7, line=dict(width=1), color=SetColor(UG))), row=1, col=3)
        fig.update_xaxes(title_text="Sigma 3 (Mpa)",titlefont=dict(family='Rockwell',size=15,color='lightgrey'), range=[-10, 30], row=1, col=3)
        fig.update_yaxes(title_text="Sigma 1 (Mpa)",titlefont=dict(family='Rockwell',size=15,color='lightgrey'), range=[0, 350], row=1, col=3)

        fig.add_trace(go.Histogram(y=stress.s1, name= UCSUG(UG), xaxis='x2', yaxis='y2', opacity=0.6, marker_color=SetColor(UG)), row=1, col=1)
        
        # Add trace
        fig.add_trace(
            go.Scatter(x=(stress.x-17000)/1667, y=(stress.y-112000)/909, mode='markers', showlegend=False,
                       marker=dict(color=SetColor(UG), size=10, opacity=0.3, line=dict(width=1, color='DarkSlateGrey')), xaxis='x3', yaxis='y3'), row=2, col=3)
        # Constants, obtenidos de las propiedades de la imagen..
        img_width = 11342
        img_height = 12431
        scale_factor = 0.0002
        
        fig.add_trace(go.Table(columnorder = [1,2], columnwidth = [50,50], header=dict(values=["<b>Parameter", "<b>Value"],
            font=dict(size=10), line_color='darkslategray', fill_color='lightskyblue',
            align="center"),
        cells=dict(
            values=[1,2], fill_color='lightcyan',
            align = "center")), row=2, col=1)
        
    fig.update_xaxes(title_text="Histogram S1 (Mpa)",titlefont=dict(family='Rockwell',size=15,color='lightgrey'), range=[30, 0], mirror=True, row=1, col=1)
    fig.update_yaxes(range=[0, 350], showticklabels=False, row=1, col=1)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    fig.layout.update(title_text="Plot Awards",width = 800, height=800)
    # Add images
    fig.layout.update(
        images=[
            go.layout.Image(
                source="Planta_EscondidaNorte_EOY23.png",
                xref="x3",
                yref="y3",
                x=0,
                y=3,
                sizex=img_width * scale_factor,
                sizey=img_height * scale_factor,
                sizing="stretch",
                opacity=0.7,
                layer="below")])
    fig.layout.update(
        yaxis3 = dict(scaleanchor = "y3", scaleratio = 1, range=[0.5, 3], domain=[0, 0.45], showgrid=False,showticklabels=False),
        xaxis3 = dict(scaleanchor = "x3", scaleratio = 1, range=[0, 2.25], domain=[0.4, 0.85], showgrid=False, showticklabels=False))

    iplot(fig)
    
origin = widgets.Dropdown(
    options=["TODOLIX", "PRIM", "PRIMSEC", "SEC"],
    value='TODOLIX',
    description='Select Chart:',
)

## definimos el widget la cual grafica cambiando la variable de la función grafico.


display(origin)
output = widgets.Output()
display(output)

def on_change(change):
    with output:
        if change['type'] == 'change' and change['name'] == 'value':
            clear_output()
            plotpro(change['new'])

origin.observe(on_change)        
        


Dropdown(description='Select Chart:', options=('TODOLIX', 'PRIM', 'PRIMSEC', 'SEC'), value='TODOLIX')

Output()